# Glasgow Traffic Analysis

## Import libraries

In [10]:
import pandas as pd
import json
import requests

## Retreive Location Data from API

In [2]:
page = requests.get(
    'https://gcc.azure-api.net/datextraffic/locations?format=json')
locations = page.json()


## Retieve Location data from Response 

In [4]:
LogicalModel = locations['d2LogicalModel']
MeasurementSites = LogicalModel['payloadPublication']['measurementSiteTable']['measurementSiteRecord']

for site in MeasurementSites:
    id = site['@id']
    location = site['measurementSiteLocation']['tpeglinearLocation']
    toLocation = location['to']
    fromLocation = location['from']
    toLocationLat = toLocation['pointCoordinates']['latitude']
    toLocationLong = toLocation['pointCoordinates']['longitude']
    toLocationName = toLocation['name']['descriptor']['value']
    fromLocationLat = fromLocation['pointCoordinates']['latitude']
    fromLocationLong = fromLocation['pointCoordinates']['longitude']
    fromLocationName = fromLocation['name']['descriptor']['value']
    #print(id, fromLocationLat, fromLocationLong, fromLocationName, toLocationLat, toLocationLong, toLocationName)


## Get Movement Data from API

In [21]:
page = requests.get('https://gcc.azure-api.net/traffic/v1/movement/now/?page=10000')
movement = page.json()


## Retrieve Movement Data from Response 

In [20]:
allsites=[]
for observation in movement:
    upd = observation['timestamp']
    site = observation['site']
    siteId = site['siteId']
    fromLoc = site['from']
    fromDesc = fromLoc['description']
    fromLat= fromLoc['lat']
    fromLong= fromLoc['long']
    flow= observation['flow']
    conc= observation['concentration']
    #print(upd,siteId, fromDesc, flow, conc)
    obs= {'updTime':upd,'SiteId':siteId, 'Desc':fromDesc, 'Flow':flow ,'Concentration': conc}
    allsites.append(obs)

df = pd.DataFrame(allsites)

import csv

with open('movement.csv','w') as f:
    w = csv.DictWriter(f, allsites[0].keys())
    w.writerows(allsites)

    


## Loop through Pages  of Current Data

In [41]:
import time
import datetime

dataReturned = True
getpage = 1
allsites=[]
while dataReturned:
    page = requests.get('https://gcc.azure-api.net/traffic/v1/movement/now/?page='+str(getpage))
    movement = page.json()
    if not movement:
        dataReturned = False
    else:
        getpage = getpage +1
        print ("Collecting data ,page ", str(getpage-1))
        for observation in movement:
            #upd = observation['timestamp']
            site = observation['site']
            siteId = site['siteId']
            fromLoc = site['from']
            fromDesc = fromLoc['description']
            fromLat= fromLoc['lat']
            fromLong= fromLoc['long']
            flow= observation['flow']
            conc= observation['concentration']
            #print(upd,siteId, fromDesc, flow, conc)
      
            obs= {'updTime':upd,'SiteId':siteId, 'Desc':fromDesc, 'Flow':flow ,'Concentration': conc}
            allsites.append(obs)
        time.sleep(5)   
df = pd.DataFrame(allsites)

import csv

with open('movement.csv','w') as f:
    w = csv.DictWriter(f, allsites[0].keys())
    w.writerows(allsites)


TypeError: string indices must be integers

In [42]:
movement


{'statusCode': 403,
 'message': 'Out of call volume quota. Quota will be replenished in 4.01:05:02.'}